In [1]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("Kaludi/food-category-classification-v2.0")
model = AutoModelForImageClassification.from_pretrained("Kaludi/food-category-classification-v2.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset

ds = load_dataset("Kaludi/food-category-classification-v2.0")

Resolving data files:   0%|          | 0/1200 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/302 [00:00<?, ?it/s]

In [4]:
from PIL import Image

def preprocess_images_and_labels(examples):
    # Convert images to RGB format or RGBA if they have transparency
    examples["image"] = [
        img.convert("RGB")
        for img in examples["image"]
    ]

    inputs = processor(images=examples["image"], return_tensors="pt")

    inputs["label"] = examples["label"]
    return inputs

ds = ds.with_transform(preprocess_images_and_labels)

In [5]:
print(processor.image_mean)
print(processor.image_std)

[0.485, 0.456, 0.406]
[0.229, 0.224, 0.225]


In [6]:
from torch.utils.data import DataLoader

# Create a DataLoader for the validation dataset
val_loader = DataLoader(ds["validation"], batch_size=16)

In [7]:
import torch
from sklearn.metrics import accuracy_score

def evaluate_model_batch(model, dataloader):
    all_preds = []
    all_labels = []

    for batch in dataloader:
        pixel_values = batch["pixel_values"].to('cuda') if torch.cuda.is_available() else batch["pixel_values"]
        labels = batch["label"].to('cuda') if torch.cuda.is_available() else batch["label"]

        with torch.no_grad():
            outputs = model(pixel_values=pixel_values)
            preds = torch.argmax(outputs.logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy



In [8]:
# Apply transformations to the dataset
ds = ds.with_transform(preprocess_images_and_labels)

# Create a DataLoader for validation split
val_loader = DataLoader(ds["validation"], batch_size=16)

In [9]:
accuracy = evaluate_model_batch(model, val_loader)
print(f"Accuracy: {accuracy}")


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Accuracy: 0.9633333333333334
